## Author: Juan Nembaopit

# (RAG) - Vector Database

## Objective

Tujuan dari kode ini adalah untuk memproses data FAQ yang disimpan dalam file CSV, memecah jawabannya menjadi potongan-potongan teks kecil, mengubah setiap potongan menjadi vektor embedding menggunakan model OpenAI terbaru, lalu menyimpan hasil embedding beserta metadata ke dalam database MongoDB Atlas agar data tersebut dapat dicari secara semantik dengan cepat dan efisien, misalnya untuk membangun chatbot FAQ yang cerdas seperti ASTRAX.

## Import Library


In [6]:
import requests
from pypdf import PdfReader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from pymongo import MongoClient
from langchain.vectorstores import MongoDBAtlasVectorSearch
import os
from dotenv import load_dotenv
from langchain_community.document_loaders import CSVLoader

## Loading env dan Data

In [7]:
# Load Environment Variables
load_dotenv(override=True)
MONGODB_URI = os.getenv("MONGO_URI")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

insight: Mengambil kredensial dari file .env untuk keamanan

In [10]:
# Load CSV Dataset
loader = CSVLoader(
    file_path="../../data/faq_combined.csv",
    content_columns=["Answer"],
    metadata_columns=["Question"],
    csv_args={'delimiter': ','}
)
documents = loader.load()

insight:
- Memuat file CSV menggunakan CSVLoader

- Kolom "Answer" sebagai konten utama

- Kolom "Question" sebagai metadata

- Menggunakan delimiter koma

## Splitting (chunking)

In [11]:
# Optimized Text Splitting
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,  # Ukuran chunk lebih kecil untuk presisi
    chunk_overlap=100,  # Overlap untuk menjaga konteks
    separators=["\n\n", "\n", "(?<=\. )", " "],
    length_function=len
)
chunks = text_splitter.split_documents(documents)

insight:
- Memecah dokumen menjadi chunk 500 karakter dengan overlap 100 karakter

- Menggunakan separators hierarkis untuk mempertahankan konteks

## Embedding

In [12]:
# Initialize Embeddings dengan model terbaru
embeddings = OpenAIEmbeddings(
    model="text-embedding-3-small",
    openai_api_key=OPENAI_API_KEY
)

Insight:
- Menggunakan model embedding terbaru OpenAI (text-embedding-3-small)

- Menghasilkan vektor 1536 dimensi untuk representasi semantik

- Model yang cost-effective dengan performa tinggi

## Load to MongoDB

In [13]:
# MongoDB Connection
client = MongoClient(MONGODB_URI)
collection = client['Astrax_db']['Astrax']

inisight:
- Membuat koneksi ke cluster MongoDB Atlas

- Mengakses database Astrax_db dan koleksi Astrax

In [14]:
collection.delete_many({})

# Vector Store Setup
vector_store = MongoDBAtlasVectorSearch.from_documents(
    documents=chunks,
    embedding=embeddings,
    collection=collection,
    index_name="vector_index"
)

print("Data berhasil diunggah ke MongoDB Atlas!")
print(f"Jumlah dokumen: {len(documents)}")
print(f"Jumlah chunks: {len(chunks)}")

Data berhasil diunggah ke MongoDB Atlas!
Jumlah dokumen: 209
Jumlah chunks: 296


insight:
- Menyimpan chunk dokumen + embedding ke MongoDB Atlas

- Menggunakan index vector_index untuk pencarian vektor

- Mengimplementasikan HNSW algorithm untuk ANN search